In [2]:
import pandas as pd
import numpy as np
import datetime
import re
#from rapidfuzz import process, fuzz
#from collections import defaultdict, Counter
#import itertools
import geopandas as gpd
from shapely.geometry import Point

In [28]:
df = pd.read_csv('./dataprog/airbnb-listings.csv')
print(len(list(df.columns)))
print(df.columns)

/tmp/ipykernel_123702/2483350946.py:1: DtypeWarning: Columns (51,53,54,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataprog/airbnb-listings.csv')


59
Index(['id', 'host_since', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_b

In [29]:
df = df.drop(columns=['last_searched', 'requires_license', 'region_id',
       'region_name', 'region_parent_id', 'region_parent_name',
       'region_parent_parent_id', 'region_parent_parent_name'], errors='ignore')

print(len(list(df.columns)))
print(df.columns)
print(df.dtypes)
print(df['price'].count())
print(df['price'].str.contains(r'\$', na=False).sum())

51
Index(['id', 'host_since', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_b

In [30]:
print(df.isna().sum())

id                                                   0
host_since                                           2
host_response_time                               95780
host_response_rate                               95780
host_acceptance_rate                             65006
host_is_superhost                                20273
host_listings_count                                  2
host_total_listings_count                            2
host_has_profile_pic                                 2
host_identity_verified                               2
latitude                                             0
longitude                                            0
property_type                                        0
room_type                                            0
accommodates                                         0
bathrooms                                          478
bedrooms                                           839
beds                                              1429
amenities 

# Clean Columns

In [31]:
df = df.dropna(subset=['host_since','review_scores_rating']) # no ratings means likely no bookings => fantasy prizes or false data
print(df.isna().sum())
df = df.dropna(subset=['review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication',
                       'review_scores_communication','review_scores_location','review_scores_value'])
print(df.isna().sum())

id                                                  0
host_since                                          0
host_response_time                              55976
host_response_rate                              55976
host_acceptance_rate                            27076
host_is_superhost                               18130
host_listings_count                                 0
host_total_listings_count                           0
host_has_profile_pic                                0
host_identity_verified                              0
latitude                                            0
longitude                                           0
property_type                                       0
room_type                                           0
accommodates                                        0
bathrooms                                         225
bedrooms                                          351
beds                                              759
amenities                   

In [32]:
parse_date = pd.Timestamp('2025-01-21')
df['host_since_date'] = pd.to_datetime(df['host_since'])
df['host_for_days'] = (parse_date - df['host_since_date']).dt.days + 1
df['host_for_log'] = np.log(df['host_for_days'])

df['host_listings_log'] = np.log(df['calculated_host_listings_count'] + 1)
df['host_listings_homes_log'] = np.log(df['calculated_host_listings_count_entire_homes'] + 1)
df['host_listings_private_log'] = np.log(df['calculated_host_listings_count_private_rooms'] + 1)
df['host_listings_shared_log'] = np.log(df['calculated_host_listings_count_shared_rooms'] + 1)

In [33]:
print(df['host_response_time'].unique())
response_time_mapping = {
    'within an hour': 10,
    'within a few hours': 7,
    'within a day': 5,
    'a few days or more': 2,
    np.nan: 1  # Handle NaN or missing values
}
df['host_response_time_ord'] = df['host_response_time'].map(response_time_mapping)

['within an hour' 'within a few hours' nan 'within a day'
 'a few days or more']


In [34]:
print(df['host_response_rate'].unique())
print(df['host_acceptance_rate'].unique())
extract_perc = re.compile(r"(\d+)\%")
def extract_percentage(value):
    if pd.isna(value):
        return 0.0
    match = extract_perc.search(value)
    if match:
        return float(match.group(1)) / 100
    return 0.0
df['host_response_rate_percent'] = df['host_response_rate'].apply(extract_percentage)
df['host_acceptance_rate_percent'] = df['host_acceptance_rate'].apply(extract_percentage)
print(df['host_response_rate_percent'].unique())
print(df['host_response_rate_percent'].unique())

['100%' nan '50%' '70%' '83%' '75%' '99%' '98%' '92%' '89%' '33%' '90%'
 '78%' '0%' '80%' '67%' '29%' '96%' '82%' '60%' '95%' '30%' '94%' '97%'
 '91%' '88%' '40%' '10%' '86%' '71%' '65%' '54%' '93%' '44%' '56%' '20%'
 '42%' '17%' '73%' '34%' '25%' '63%' '13%' '43%' '57%' '59%' '87%' '84%'
 '79%' '53%' '22%' '46%' '14%' '11%' '66%' '55%' '51%' '62%' '85%' '77%'
 '23%' '38%' '68%' '16%' '81%' '64%' '39%' '36%' '74%' '5%' '72%' '76%'
 '48%' '69%' '32%' '52%' '31%' '58%' '27%' '61%' '47%' '7%' '41%' '8%'
 '18%' '37%' '26%' '28%' '15%' '21%' '4%' '9%' '24%' '49%' '45%' '6%'
 '12%' '3%' '35%' '1%' '19%']
['100%' '97%' '71%' '50%' '25%' '75%' '57%' '99%' '13%' '56%' '67%' '92%'
 '31%' '98%' '37%' '86%' '82%' '55%' '0%' '11%' '46%' '83%' '77%' '60%'
 '93%' '70%' '36%' '62%' '45%' '30%' '43%' '78%' '33%' '38%' '95%' '80%'
 '18%' '39%' nan '40%' '61%' '88%' '91%' '53%' '79%' '7%' '52%' '15%'
 '63%' '44%' '29%' '84%' '96%' '35%' '47%' '90%' '59%' '21%' '87%' '73%'
 '20%' '89%' '68%' '85%' '76%' '

In [35]:
print(df['host_is_superhost'].unique())
print(df['host_has_profile_pic'].unique())
print(df['host_identity_verified'].unique())
print(df["has_availability"].unique())
print(df["instant_bookable"].unique())

ohe_mapping = {
    'f': 0,
    't': 1,
    np.nan: 0
}
df['is_superhost'] = df['host_is_superhost'].map(ohe_mapping)
df['is_host_has_pic'] = df['host_has_profile_pic'].map(ohe_mapping)
df['is_host_id_verified'] = df['host_identity_verified'].map(ohe_mapping)
df['is_available'] = df['has_availability'].map(ohe_mapping)
df['is_instant_bookable'] = df['instant_bookable'].map(ohe_mapping)
print(df['is_superhost'].unique())

['f' 't' nan]
['t' 'f']
['t' 'f']
['t' nan 'f']
['t' 'f']
[0 1]


In [36]:
df['listings_count'] = df['host_listings_count'].fillna(0).astype(float).map(int)
df['listings_count_total'] = df['host_total_listings_count'].fillna(0).astype(float).map(int)
df['bathrooms'] = df['bathrooms'].fillna(0)
df['bedrooms'] = df['bedrooms'].fillna(0).map(int)
df['beds'] = df['beds'].fillna(0).map(int)
df['minimum_minimum_nights'] = df['minimum_minimum_nights'].map(int)
df['maximum_minimum_nights'] = df['maximum_minimum_nights'].map(int)
df['minimum_maximum_nights'] = df['maximum_maximum_nights'].map(int)

In [37]:
print(df['room_type'].unique())
room_type_mapping = {
    'Private room': 'private',
    'Entire home/apt': 'entire',
    'Hotel room': 'hotel',
    'Shared room': 'shared'
}
df['type'] = df['room_type'].map(room_type_mapping)
df_room_type_ohe = pd.get_dummies(df['type'], prefix='type')
df = pd.concat([df, df_room_type_ohe], axis=1)

['Private room' 'Entire home/apt' 'Hotel room' 'Shared room']


In [38]:
print(df['property_type'].unique())
extract_property_in = re.compile(r"\bin\s([\w\s\/-]+)")
extract_property_entire = re.compile(r"\bEntire\s([\w\s\/-]+)")
def extract_property(value):
    in_match = extract_property_in.search(value)
    if in_match:
        return in_match.group(1).lower()
    entire_match = extract_property_entire.search(value)
    if entire_match:
        return entire_match.group(1).lower()
    return value.lower()

df['property'] = df['property_type'].apply(extract_property)
property_mapping = {
    'bed and breakfast': 'bnb',
    'loft': 'apartment',
    'condo': 'apartment',
    'guest suite': 'suite',
    'rental unit': 'apartment',
    'serviced apartment': 'apartment',
    'townhouse': 'house',
    'boutique hotel': 'hotel',
    'private room': 'room',
    'boat': 'other',
    'houseboat': 'other',
    'casa particular': 'bnb',
    'hut': 'house',
    'farm stay': 'farm',
    'villa': 'house',
    'place': 'apartment',
    'bungalow': 'house',
    'aparthotel': 'hotel',
    'tiny home': 'house',
    'vacation home': 'house',
    'cabin': 'house',
    'barn': 'farm',
    'earthen home': 'house',
    'cottage': 'house',
    'nature lodge': 'house',
    'chalet': 'house',
    'cycladic home': 'house',
    'dome': 'house',
    'yurt': 'tent',
    'in-law': 'house',
    'pension': 'bnb',
    'minsu': 'other',
    'home/apt': 'apartment',
    'cycladic house': 'house',
    'bus': 'other',
    'holiday park': 'other',
    'tipi': 'tent',
    "shepherd’s hut": 'other',
    'shepherd': "other",
    'heritage hotel': 'hotel',
    'lighthouse': 'other',
    'ryokan': 'hotel',
    'dammuso': 'other',
    'riad': 'other',
    'ice dome': 'other',
    'kezhan': 'hotel',
    'trullo': 'house',
    'train': 'other',
    'ranch': 'farm',
    'home': 'house',
    'camper/rv': 'campsite',
    'religious building': 'other',
    'tower': 'other',
    'cave': 'other',
    'island': 'other',
    'plane': 'other',
    'shipping container': 'other',
    'castle': 'other',
    'windmill': 'other',
    'shared room': 'room',
    'resort': 'other',
    'floor': 'other',
    'treehouse': 'other',
    'dorm': 'other'
}
df['property'] = df['property'].map(property_mapping).fillna(df['property'])
print(df['property'].unique())
df_property_type_ohe = pd.get_dummies(df['property'], prefix='type')
df = pd.concat([df, df_property_type_ohe], axis=1)

['Room in hotel' 'Private room in bed and breakfast'
 'Private room in guesthouse' 'Entire condo' 'Private room in guest suite'
 'Entire home' 'Entire rental unit' 'Entire serviced apartment'
 'Entire guesthouse' 'Boat' 'Private room in townhouse' 'Entire townhouse'
 'Private room in condo' 'Private room in rental unit'
 'Room in boutique hotel' 'Private room in home' 'Entire loft'
 'Private room' 'Houseboat' 'Private room in houseboat' 'Camper/RV'
 'Private room in loft' 'Private room in casa particular'
 'Private room in boat' 'Private room in hut' 'Private room in farm stay'
 'Entire villa' 'Entire place' 'Private room in bungalow'
 'Room in aparthotel' 'Tiny home' 'Room in bed and breakfast'
 'Shared room in houseboat' 'Entire vacation home' 'Room in hostel'
 'Private room in serviced apartment' 'Casa particular'
 'Shared room in hostel' 'Entire cabin' 'Room in serviced apartment'
 'Entire guest suite' 'Barn' 'Private room in hostel'
 'Private room in earthen home' 'Private room in

In [39]:
print(df[df['minimum_nights'] > 90].shape)
print(df[df['maximum_nights'] > 1124].shape)
print(df[df['maximum_nights'] > 1125].shape)
df = df[df['minimum_nights'] <= 90]
df = df[df['maximum_nights'] <= 1125]
print(df.shape)

(8813, 86)
(166474, 86)
(39, 86)
(580156, 86)


In [40]:
df["availability_30_percent"] = df["availability_30"] / 30
df["availability_60_percent"] = df["availability_60"] / 60
df["availability_90_percent"] = df["availability_90"] / 90
df["availability_365_percent"] = df["availability_365"] / 365

In [41]:
df["price_per_night_dollar"] = df['price'].str.replace(r'\$', '', regex=True).replace(r',', '', regex=True).map(float)

In [42]:
df['amenities_array'] = df['amenities'].apply(eval)
df['total_amenities'] = df['amenities_array'].count()

In [43]:
mapping_table = {
    r'(?i)Free.*parking.*premises': 'Free Parking',
    r'(?i)Free.*garage.*premises': 'Free Parking',
    r'(?i)Free.*carport.*premises': 'Free Parking',
    r'(?i)Paid.*garage.*premises': 'Paid Parking',
    r'(?i)Paid.*parking.*premises': 'Paid Parking',
    r'(?i)Paid.*carport.*premises': 'Paid Parking',
    r'(?i)wifi': 'Internet',
    r'(?i)ethernet': 'Internet',
    r'(?i)body soap': 'Bath Supplies',
    r'(?i)shampoo': 'Bath Supplies',
    r'(?i)shower gel': 'Bath Supplies',
    r'(?i)conditioner': 'Bath Supplies',
    r'(?i)cooking': 'Kitchen Supplies',
    r'(?i)dishes and silverware': 'Kitchen Supplies',
    r'(?i)Dishwasher': 'Kitchen Supplies',
    r'(?i)Wine glasses': 'Kitchen Supplies',
    r'(?i)Baking sheet': 'Kitchen Supplies',
    r'(?i)Barbecue utensils': 'Kitchen Supplies',
    r'(?i)outdoor': 'outdoor',
    r'(?i)hdtv': 'TV',
    r'TV': 'TV',
    r'(?i)kitchen': 'Kitchen',
    r'(?i)coffee': 'Coffee',
    r'AC': 'Air conditioning',
    r'(?i)Air conditioning': 'Air conditioning',
    r'EV': 'EV Charger',
    r'(?i)stove': 'Stove',
    r'(?i)oven': 'Oven',
    r'(?i)high chair': 'Child friendly',
    r'(?i)children': 'Child friendly',
    r'(?i)baby': 'Child friendly',
    r'(?i)Crib': 'Child friendly',
    r'(?i)washer': 'Washer',
    r'(?i)dryer': 'Dryer',
    r'(?i)game': 'Games',
    r'(?i)grill': 'Grill',
    r'(?i)hot tub': 'Hot Tub',
    r'(?i)sauna': 'Sauna',
    r'(?i)pool': 'Pool',
    r'(?i)heating': 'Heating',
    r'(?i)view': 'View',
    r'(?i)refrigerator': 'Refrigerator',
    r'(?i)sound system': 'Sound System',
    r'(?i)housekeeping': 'Housekeeping',
    r'(?i)exercise': 'Exercise',
    r'(?i)gym': 'Exercise',
    r'(?i)exercise': 'Exercise',
    r'(?i)squash': 'Exercise',
    r'(?i)tennis': 'Exercise',
    r'(?i)court': 'Exercise',
    r'(?i)skate': 'Exercise',
    r'(?i)climbing': 'Exercise',
    r'Ski': 'Exercise',
    r'(?i)bowling': 'Exercise',
    r'(?i)batting': 'Exercise',
    r'(?i)bikes?': 'Exercise',
    r'(?i)Hockey': 'Exercise',
    r'(?i)yoga': 'Exercise',
    r'(?i)clothing storage': 'Clothing storage',
    r'(?i)security': 'Safety features',
    r'(?i)Gated': 'Safety features',
    r'(?i)keybad': 'Safety features',
    r'(?i)safe': 'Safety features',
    r'(?i)fire pit': 'Fireplace',
    r'(?i)fireplace': 'Fireplace',
    r'(?i)beach.*access': 'Beach',
    r'(?i)bar': 'Bar',
}

def map_amenities(amenities, mapping):
    replaced = []
    for amenity in amenities:
        replaced_amenity = amenity
        for pattern, replacement in mapping.items():
            if re.search(pattern, amenity):
                #replaced_amenity = 
                replaced.append(replacement)
                break
        
    return replaced

df['amenities_array_mapped'] = df['amenities_array'].apply(lambda x: map_amenities(x, mapping_table))

In [44]:
dummies = pd.get_dummies(df['amenities_array_mapped'].explode(), prefix='amenity').groupby(level=0).sum()
df = pd.concat([df, dummies], axis=1)
print(df.head())

                   id  host_since  host_response_time host_response_rate  \
0  888038298563003925  2019-11-22      within an hour               100%   
1            50666229  2021-05-10      within an hour               100%   
2              933523  2013-02-10      within an hour               100%   
4  656222143862971193  2015-07-23  within a few hours               100%   
5            20259345  2014-09-27      within an hour               100%   

  host_acceptance_rate host_is_superhost  host_listings_count  \
0                 100%                 f                  9.0   
1                 100%                 t                  2.0   
2                  97%                 t                  1.0   
4                 100%                 f                  1.0   
5                 100%                 t                  2.0   

   host_total_listings_count host_has_profile_pic host_identity_verified  ...  \
0                        9.0                    t                      

In [62]:
"""
#print(df['amenities'].unique())
flattened_tags = list(itertools.chain.from_iterable(df['amenities_array_mapped']))
tag_counts = Counter(flattened_tags)
tags_to_fuzz = [tag for tag, count in tag_counts.items() if count >= 10]
print(len(tags_to_fuzz))
def cluster_tags(tags, threshold=80):
    clusters = {}
    for tag in tags:
        # Find the closest match in existing clusters
        result = process.extractOne(tag, clusters.keys(), scorer=fuzz.ratio)
        if result:  # Check if a match was found
            match = result[0]  # Extract the matched string
            score = result[1]  # Extract the similarity score
            if score >= threshold:
                clusters[match].append(tag)
            else:
                clusters[tag] = [tag]
        else:
            clusters[tag] = [tag]
    return clusters

clusters = cluster_tags(tags_to_fuzz, threshold=70)
print(f"Clusters formed: {len(clusters)}")
for key, cluster in clusters.items():
    print(f"Cluster: {key}, Tags: {cluster}")
    if len(cluster) > 5:  # Limit the output
        break
canonical_mapping = {tag: cluster[0] for cluster in clusters.values() for tag in cluster}

print(f"Canonical mapping size: {len(canonical_mapping)}")
print(f"Sample mappings: {list(canonical_mapping.items())[:10]}")

missing_tags = [tag for tag in flattened_tags if tag not in canonical_mapping]
print(f"Tags missing in canonical_mapping: {len(missing_tags)}")
print(f"Sample missing tags: {missing_tags[:10]}")

standardized_tags = [canonical_mapping.get(tag, tag) for tag in flattened_tags]

print(f"Length of standardized_tags: {len(standardized_tags)}")
print(f"Sample standardized tags: {standardized_tags[:10]}")

standardized_counts = Counter(standardized_tags)

print(f"Standardized counts: {standardized_counts}")

tag_counts_df = pd.DataFrame.from_dict(standardized_counts, orient='index', columns=['count']).reset_index()
tag_counts_df.rename(columns={'index': 'tag'}, inplace=True)

print(tag_counts_df)
"""

168
Clusters formed: 122
Cluster: Paid Parking, Tags: ['Paid Parking']
Cluster: Bath Supplies, Tags: ['Bath Supplies', 'Kitchen Supplies']
Cluster: Coffee, Tags: ['Coffee']
Cluster: Carbon monoxide alarm, Tags: ['Carbon monoxide alarm']
Cluster: Internet, Tags: ['Internet']
Cluster: Smoke alarm, Tags: ['Smoke alarm']
Cluster: Extra pillows and blankets, Tags: ['Extra pillows and blankets']
Cluster: Safe, Tags: ['Safe']
Cluster: Luggage dropoff allowed, Tags: ['Luggage dropoff allowed']
Cluster: Dryer, Tags: ['Dryer']
Cluster: Elevator, Tags: ['Elevator']
Cluster: EV Charger, Tags: ['EV Charger']
Cluster: Cleaning available during stay, Tags: ['Cleaning available during stay']
Cluster: Air conditioning, Tags: ['Air conditioning']
Cluster: Hangers, Tags: ['Hangers']
Cluster: Essentials, Tags: ['Essentials']
Cluster: Heating, Tags: ['Heating']
Cluster: TV, Tags: ['TV']
Cluster: First aid kit, Tags: ['First aid kit']
Cluster: Fire extinguisher, Tags: ['Fire extinguisher']
Cluster: High cha

In [45]:
df = df.drop(columns=["host_for_days","host_since_date","host_since","host_response_time","host_response_rate","host_is_superhost",
                 "host_listings_count","host_total_listings_count","host_has_profile_pic","host_acceptance_rate","host_identity_verified",
                 "room_type","type","property","property_type","has_availability","availability_30","availability_60","availability_90",
                 "availability_365","instant_bookable","calculated_host_listings_count","calculated_host_listings_count_entire_homes",
                 "calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms", "price", "amenities",
                 "amenities_array","amenities_array_mapped"], errors='ignore')

print(list(df.columns))
df.head()

['id', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'country', 'city', 'host_for_log', 'host_listings_log', 'host_listings_homes_log', 'host_listings_private_log', 'host_listings_shared_log', 'host_response_time_ord', 'host_response_rate_percent', 'host_acceptance_rate_percent', 'is_superhost', 'is_host_has_pic', 'is_host_id_verified', 'is_available', 'is_instant_bookable', 'listings_count', 'listings_count_total', 'type_entire', 'type_hotel', 'type_private', 'type_shared', 'type_apartment', 'type

,id,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,minimum_minimum_nights,...,amenity_Pool,amenity_Refrigerator,amenity_Safety features,amenity_Sauna,amenity_Sound System,amenity_Stove,amenity_TV,amenity_View,amenity_Washer,amenity_outdoor
0,888038298563003925,52.370110,4.879090,3,1.0,1,3,1,365,1,...,0,0,1,0,0,0,1,0,0,0
1,50666229,52.359452,4.861793,4,1.0,2,5,4,365,2,...,0,1,0,0,0,0,1,2,0,0
2,933523,52.356197,4.991435,4,1.5,2,2,3,30,2,...,0,1,0,0,0,0,1,4,0,2
4,656222143862971193,52.372422,4.882609,2,1.0,1,1,3,10,2,...,0,1,0,0,1,1,1,1,1,2
5,20259345,52.367090,4.865750,2,1.5,1,1,1,10,1,...,0,1,0,0,0,0,1,2,0,0


In [46]:
print(df["country"].unique())
country_mapping = {
    'the-netherlands': 'netherlands',
    'ireland': 'ireland-and-northern-ireland',
}

df['country'] = df['country'].map(country_mapping).fillna(df['country'])

['the-netherlands' 'belgium' 'greece' 'spain' 'italy' 'germany' 'france'
 'united-kingdom' 'denmark' 'ireland' 'switzerland' 'turkey' 'portugal'
 'norway' 'czech-republic' 'latvia' 'sweden' 'austria' 'malta']


In [47]:
output_file_path = "airbnb_cleaned.csv"
df.to_csv(output_file_path, index=False)

print(f"Cleaned data saved to {output_file_path}")

Cleaned data saved to airbnb_cleaned.csv


# Consolidate with pois in the area

In [3]:
pois = gpd.read_file("./dataprog/consolidated_pois.geojson")
df = pd.read_csv("./airbnb_cleaned.csv")
country_crs_mapping = {
    'albania': 32634,
    'kosovo': 32634,
    'andorra': 32631,
    'latvia': 32635,
    'austria': 32633,
    'liechtenstein': 32632,
    'azores': 32626,
    'lithuania': 32635,
    'belarus': 32635,
    'luxembourg': 32631,
    'belgium': 32631,
    'macedonia': 32634,
    'bosnia-herzegovina': 32633,
    'malta': 32633,
    'bulgaria': 32635,
    'moldova': 32635,
    'croatia': 32633,
    'monaco': 32632,
    'cyprus': 32636,
    'montenegro': 32634,
    'czech-republic': 32633,
    'netherlands': 32631,
    'denmark': 32632,
    'norway': 32632,
    'estonia': 32635,
    'poland': 32634,
    'faroe-islands': 32629,
    'portugal': 32629,
    'finland': 32635,
    'romania': 32635,
    'france': 32631,  # Defaulting to Western France
    'russia': 32636,  # Defaulting to Western Russia
    'georgia': 32638,
    'serbia': 32634,
    'germany': 32632,  # Defaulting to Western Germany
    'slovakia': 32633,
    'greece': 32634,  # Defaulting to Western Greece
    'slovenia': 32633,
    'guernsey-jersey': 32630,
    'spain': 32630,  # Defaulting to Western Spain
    'hungary': 32634,
    'sweden': 32633,  # Defaulting to Southern Sweden
    'iceland': 32627,
    'switzerland': 32632,
    'ireland-and-northern-ireland': 32629,
    'turkey': 32636,
    'isle-of-man': 32630,
    'ukraine': 32636,
    'italy': 32632,  # Defaulting to Western Italy
    'united-kingdom': 32630
}
global_crs = "EPSG:4326"

In [4]:
def convert_to_geodf(df, lat_col, lon_col, crs="EPSG:4326"):
    # Create a geometry column using shapely's Point
    df['geometry'] = df.apply(lambda row: Point(row[lon_col], row[lat_col]), axis=1)
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    # Set the Coordinate Reference System (CRS)
    gdf.set_crs(crs, inplace=True)
    return gdf

accommodations = convert_to_geodf(df, lat_col='latitude', lon_col='longitude')

In [26]:
"""
# Define the radii for testing
radii = [500, 1000, 5000]

# Prepare the result DataFrame
result = pd.DataFrame()

# Group accommodations by country
for country, subset_accommodations in accommodations.groupby('country'):
    # Get the corresponding EPSG for the country
    target_epsg = country_crs_mapping.get(country.lower(), 3035)  # Default to EPSG:3035
    print(target_epsg)
    
    # Transform accommodations and POIs to the appropriate CRS
    subset_accommodations = subset_accommodations.to_crs(epsg=target_epsg)
    country_pois = pois[pois['country'] == country].copy()
    country_pois = country_pois.to_crs(epsg=target_epsg)
    
    # Process accommodations for this country
    subset_result = subset_accommodations.copy()

    for radius in radii:
        subset_result[f'buffer_{radius}m'] = subset_accommodations["geometry"].buffer(radius)
    
    for radius in radii:
        if subset_result[f'buffer_{radius}m'].isna().any():
            print(f"Some buffers are None for radius {radius}m in {country}")
        
        # Use spatial index to find POIs within the radius
        enriched_data = []
        for idx, accommodation in subset_result.iterrows():
            # Get the buffer geometry
            buffer_geom = accommodation[f'buffer_{radius}m']
            
            # Query the spatial index for possible matches
            possible_matches_index = list(country_pois.sindex.query(buffer_geom))
            possible_matches = country_pois.iloc[possible_matches_index]
            
            # Filter points within the buffer
            pois_within_radius = possible_matches[possible_matches["geometry"].within(buffer_geom)]
            
            # Aggregate POI information (example: count by category)
            poi_summary = pois_within_radius['tag_value'].value_counts().to_dict()
            enriched_data.append(poi_summary)
        
        # Add enrichment data as new columns (one column per category, per radius)
        summary_df = pd.DataFrame(enriched_data).fillna(0).astype(int)
        summary_df.columns = [f'{col}_within_{radius}m' for col in summary_df.columns]
        subset_result = pd.concat([subset_result.reset_index(drop=True), summary_df.reset_index(drop=True)], axis=1)
    
    # Drop buffer columns for a cleaner result
    subset_result = subset_result.drop(columns=[f'buffer_{radius}m' for radius in radii])
    
    # Transform back to global CRS for consistency
    subset_result = subset_result.to_crs(global_crs)
    
    # Append the processed data for this country to the result DataFrame
    #subset_result = subset_result.reset_index(drop=True)
    #result = result.reset_index(drop=True)
    #overlapping_columns = set(result.columns).intersection(set(subset_result.columns))
    #print(f"Overlapping Columns: {overlapping_columns}")
    result = pd.concat([result, subset_result], ignore_index=True)

# Final result
print(result)
"""

32633
Overlapping Columns: set()
32631
Overlapping Columns: {'train-station_within_1000m', 'grave-yard_within_1000m', 'winery_within_1000m', 'retail_within_500m', 'post-office_within_500m', 'atelier_within_500m', 'beekeeper_within_500m', 'cafe_within_5000m', 'police_within_5000m', 'boat-rental_within_500m', 'clock_within_1000m', 'funeral-hall_within_500m', 'transport-station_within_500m', 'monument_within_1000m', 'monastery_within_1000m', 'handicraft_within_5000m', 'watchmaker_within_1000m', 'pub_within_500m', 'supermarket_within_500m', 'post-box_within_500m', 'commercial_within_1000m', 'recycling_within_5000m', 'geometry', 'university_within_5000m', 'commercial_within_500m', 'car-rental_within_5000m', 'grandstand_within_5000m', 'gift_within_1000m', 'taxi_within_500m', 'supermarket_within_5000m', 'tailor_within_500m', 'attraction_within_500m', 'watchmaker_within_5000m', 'transport-station_within_1000m', 'pottery_within_500m', 'locomotive_within_500m', 'viewpoint_within_5000m', 'bus-sta

KeyboardInterrupt: 

In [8]:
from concurrent.futures import ProcessPoolExecutor

def process_country(country_data):
    radii = [500, 1000, 5000]
    country, (subset_accommodations, pois) = country_data
    print(country)
    
    # Get the EPSG for the country
    # Transform accommodations and POIs to the appropriate CRS
    target_epsg = country_crs_mapping.get(country.lower(), 3035)  # Default to EPSG:3035
    subset_accommodations = subset_accommodations.to_crs(epsg=target_epsg)
    country_pois = pois.to_crs(epsg=target_epsg)
    print(target_epsg)
    
    
    # Process accommodations for this country
    subset_result = subset_accommodations.copy()
    #subset_result = subset_accommodations

    for radius in radii:
        subset_result[f'buffer_{radius}m'] = subset_accommodations["geometry"].buffer(radius)
    
    for radius in radii:
        if subset_result[f'buffer_{radius}m'].isna().any():
            print(f"Some buffers are None for radius {radius}m in {country}")
        print("Processing radius for country:",radius, country)
        
        # Use spatial index to find POIs within the radius
        enriched_data = []
        for idx, accommodation in subset_result.iterrows():
            # Get the buffer geometry
            buffer_geom = accommodation[f'buffer_{radius}m']
            
            # Query the spatial index for possible matches
            possible_matches_index = list(country_pois.sindex.query(buffer_geom))
            possible_matches = country_pois.iloc[possible_matches_index]
            
            # Filter points within the buffer
            pois_within_radius = possible_matches[possible_matches["geometry"].within(buffer_geom)]
            
            # Aggregate POI information (example: count by category)
            poi_summary = pois_within_radius['tag_value'].value_counts().to_dict()
            enriched_data.append(poi_summary)
        
        # Add enrichment data as new columns (one column per category, per radius)
        summary_df = pd.DataFrame(enriched_data).fillna(0).astype(int)
        summary_df.columns = [f'{col}_within_{radius}m' for col in summary_df.columns]
        subset_result = pd.concat([subset_result.reset_index(drop=True), summary_df.reset_index(drop=True)], axis=1)
    
    # Drop buffer columns for a cleaner result
    subset_result = subset_result.drop(columns=[f'buffer_{radius}m' for radius in radii])
    
    # Transform back to global CRS for consistency
    subset_result = subset_result.to_crs(global_crs)
    
    # Append the processed data for this country to the result DataFrame
    #subset_result = subset_result.reset_index(drop=True)
    #result = result.reset_index(drop=True)
    #overlapping_columns = set(result.columns).intersection(set(subset_result.columns))
    #print(f"Overlapping Columns: {overlapping_columns}")
    #result = pd.concat([result, subset_result], ignore_index=True)
    print("Merged: ", country)
    print(subset_result.columns)
    
    return subset_result

# Prepare country data for parallel processing
country_data = [
    (country, (
        accommodations[accommodations['country'] == country],
        pois[pois['country'] == country]
    ))
    #for country in accommodations['country'].unique()
    for country in ["united-kingdom"]
]

# Process in parallel
with ProcessPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(process_country, country_data))

print("Finished processing")

united-kingdom
32630
Processing radius for country: 500 united-kingdom
Processing radius for country: 1000 united-kingdom
Processing radius for country: 5000 united-kingdom
Merged:  united-kingdom
Index(['id', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       ...
       'glassblower_within_5000m', 'religious_within_5000m',
       'mint_within_5000m', 'pagoda_within_5000m', 'gallows_within_5000m',
       'beekeeper_within_5000m', 'anchor_within_5000m',
       'locomotive_within_5000m', 'station_within_5000m',
       'stage_within_5000m'],
      dtype='object', length=428)
Finished processing


In [ ]:
#better memory-profile
from concurrent.futures import ProcessPoolExecutor

def process_country_better(country_data):
    radii = [500, 1000, 5000]
    country, (subset_accommodations, pois) = country_data
    print(country)
    
    # Get the EPSG for the country
    # Transform accommodations and POIs to the appropriate CRS
    target_epsg = country_crs_mapping.get(country.lower(), 3035)  # Default to EPSG:3035
    subset_accommodations = subset_accommodations.to_crs(epsg=target_epsg)
    pois = pois.to_crs(epsg=target_epsg)
    
    
    # Process accommodations for this country
    #subset_result = subset_accommodations.copy()
    #subset_result = subset_accommodations
    enriched_data = []

    for radius in radii:
        subset_result[f'buffer_{radius}m'] = subset_accommodations["geometry"].buffer(radius)
    
    for radius in radii:
        if subset_result[f'buffer_{radius}m'].isna().any():
            print(f"Some buffers are None for radius {radius}m in {country}")
        print("Processing radius for country:",radius, country)
        sindex = pois.sindex  # Spatial index for POIs
        
        # Use spatial index to find POIs within the radius
        for idx, accommodation in subset_result.iterrows():
            # Get the buffer geometry
            buffer_geom = accommodation[f'buffer_{radius}m']
            
            # Query the spatial index for possible matches
            possible_matches_index = list(sindex.query(buffer_geom))
            possible_matches = pois.iloc[possible_matches_index]
            pois_within_radius = possible_matches[possible_matches["geometry"].within(buffer_geom)]
            
            # Aggregate POI information (example: count by category)
            poi_summary = pois_within_radius['tag_value'].value_counts().to_dict()
           enriched_data.append({**{'id': accommodation['id']}, **poi_summary})
        
    # Add enrichment data as new columns (one column per category, per radius)
    enriched_df = pd.DataFrame(enriched_data).fillna(0)
    for radius in radii:
        sparse_cols = [col for col in enriched_df.columns if col != 'id']
        enriched_df.columns = ['id'] + [f'{col}_within_{radius}m' for col in sparse_cols]
    summary_df.columns = [f'{col}_within_{radius}m' for col in summary_df.columns]
    subset_result = pd.concat([subset_result.reset_index(drop=True), summary_df.reset_index(drop=True)], axis=1)
    
    # Drop buffer columns for a cleaner result
    subset_result = subset_result.drop(columns=[f'buffer_{radius}m' for radius in radii])
    
    # Transform back to global CRS for consistency
    subset_result = subset_result.to_crs(global_crs)
    
    # Append the processed data for this country to the result DataFrame
    #subset_result = subset_result.reset_index(drop=True)
    #result = result.reset_index(drop=True)
    #overlapping_columns = set(result.columns).intersection(set(subset_result.columns))
    #print(f"Overlapping Columns: {overlapping_columns}")
    #result = pd.concat([result, subset_result], ignore_index=True)
    print("Merged: ", country)
    print(subset_result.columns)
    
    return subset_result

# Prepare country data for parallel processing
country_data = [
    (country, (
        accommodations[accommodations['country'] == country],
        pois[pois['country'] == country]
    ))
    #for country in accommodations['country'].unique()
    for country in ["united-kingdom"]
]

# Process in parallel
with ProcessPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(process_country_better, country_data))

print("Finished processing")

In [9]:
# Combine all results into a single GeoDataFrame
final_result = pd.concat(results, ignore_index=True)

# Merge back with the original DataFrame (if necessary)
#final_result = original_df.merge(final_result, on='id', how='left')

# Final Output
print(final_result)
output_path = 'train2.csv'  # Replace with your desired file path
final_result.drop(columns=['geometry','id','name'], errors='ignore').to_csv(output_path, index=False)

print(f"GeoDataFrame exported to {output_path} without the geometry column")

                        id   latitude  longitude  accommodates  bathrooms  \
0                   117122  51.449760  -2.610000             2        2.5   
1                   176306  51.441740  -2.602970             1        1.0   
2                   307634  51.464870  -2.556900             2        1.5   
3                   391093  51.477821  -2.591995             2        1.0   
4                   399681  51.494910  -2.618790             3        1.0   
...                    ...        ...        ...           ...        ...   
59888  1236978227828303426  51.525374  -0.078516             8        2.0   
59889  1237129853877004021  51.526713  -0.192734             2        1.0   
59890  1237770824910464719  51.496290  -0.194660             4        1.0   
59891  1237869943469530141  51.514281  -0.067786             2        1.0   
59892  1238560141463687738  51.528992  -0.118654             2        1.0   

       bedrooms  beds  minimum_nights  maximum_nights  minimum_minimum_nigh

In [ ]:
has_columns = [col for col in result.columns if '_within_' in col and col.endswith('m')]

# Replace NaN with 0 only in the identified columns
result[has_columns] = result[has_columns].fillna(0).astype(int)
#result['lat'] = pd.concatresult['geometry'].y
#result['lon'] = result['geometry'].x
# Print the first few rows to verify the changes
print(result.head())
print(result.describe(include='all'))
print(list(result.columns))

In [ ]:
output_path = 'train.csv'  # Replace with your desired file path
result.drop(columns=['geometry','id','name']).to_csv(output_path, index=False)

print(f"GeoDataFrame exported to {output_path} without the geometry column")